<a href="https://colab.research.google.com/github/tallerzalan/Applied-Machine-Learning/blob/main/NNs/Exercise_2_nn_optimizers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise - optimizers and activation functions

1. Use the $\texttt{fetch_california_housing}$ data (remember to split your data into a train and test data). Use the five optimizers presented in class to train five neural networks (identival aside from the optimizer used). How well does the networks perform on the test set, as measured by MSE and MAE? Rank the optimizers.
1. Select the best optimizer and use it for this exercise. Experiment with different activation functions, including at least sigmoid, tanh, and relu. Rank the activation functions you try. 
1. Using your findings, as well as experimenting with more layers, try to minimize the test MSE.

**Note**: You may want to use https://www.tensorflow.org/api_docs/python/tf/keras/activations and https://www.tensorflow.org/api_docs/python/tf/keras/optimizers.

**See slides for more details!**

# Exercise 1

Use the $\texttt{fetch_california_housing}$ data (remember to split your data into a train and test data). Use the five optimizers presented in class to train five neural networks (identival aside from the optimizer used). How well does the networks perform on the test set, as measured by MSE and MAE? Rank the optimizers.

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Activation, Dense
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

import pandas as pd
from matplotlib import pyplot as plt

seed = 42
tf.random.set_seed(seed)

In [ ]:
X, y = fetch_california_housing(return_X_y = True)

# Use `train_test_split` to split your data into a train and a test set.
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.2,
                                                    random_state = seed
                                                    )

# Scale
scaler = StandardScaler()
Z_train = scaler.fit_transform(X_train)
Z_test = scaler.transform(X_test)

print(Z_train.shape, Z_test.shape, y_train.shape, y_test.shape)

(16512, 8) (4128, 8) (16512,) (4128,)


In [ ]:
def build_nn(activation = 'sigmoid'):
    your_regression_nn = Sequential([
        Dense(64, activation = activation, input_shape = (8,)), # input_shape = 8 as there are 8 features
        Dense(1, activation = 'linear') # linear is used for regression. 1 node since 1 output (pr. observation)
        ])

    return your_regression_nn

In [ ]:
# SGD
nn_sgd = build_nn(activation = 'sigmoid')

nn_sgd.compile(
    optimizer = 'sgd',
    loss = 'mse',
    metrics = ['mae'], # to also track MAE. MSE is "automatically" measured since it is the loss
    )

nn_sgd.fit(Z_train,
           y_train,
           epochs = 5
           )

mse, mae = nn_sgd.evaluate(Z_test,
                           y_test
                           )

print(f'Test MSE = {round(mse, 3)}, test MAE = {round(mae, 3)}.')

Epoch 1/5
516/516 [==============================] - 1s 2ms/step - loss: 0.7512 - mae: 0.6597
Epoch 2/5
516/516 [==============================] - 1s 2ms/step - loss: 0.5654 - mae: 0.5567
Epoch 3/5
516/516 [==============================] - 1s 2ms/step - loss: 0.5317 - mae: 0.5373
Epoch 4/5
516/516 [==============================] - 1s 2ms/step - loss: 0.5132 - mae: 0.5266
Epoch 5/5
129/129 [==============================] - 0s 2ms/step - loss: 0.5145 - mae: 0.5220
Test MSE = 0.514, test MAE = 0.522.


**Important note**: Remember to use "mse" as your loss function! Now, it is okay to try something else, but at least do not use cross entropy (remember that is for classification.

Go through each of the five optimizers covered in class and rank their performance on this dataset.

In [ ]:
activations = ['sigmoid', 'tanh', 'relu']
optimizers = ['sgd', 'adam', 'adagrad', 'adadelta', 'adamax', 'nadam', 'rmsprop']

results = []

for activation in activations:
  for optimizer in optimizers:
    nn = build_nn(activation)
    nn.compile(
        optimizer = optimizer,
        loss = 'mse',
        metrics = ['mae']
        )
    
    nn.fit(Z_train,
           y_train,
           epochs = 5
           )
    
    mse, mae = nn.evaluate(Z_test,
                           y_test
                           )
    
    results.append([activation, optimizer, mse, mae])

results = pd.DataFrame(results)
results.columns = ['Activation', 'Optimizer', 'MSE', 'MAE']
print(results)

Epoch 1/5
516/516 [==============================] - 1s 2ms/step - loss: 0.7645 - mae: 0.6657
Epoch 2/5
516/516 [==============================] - 1s 2ms/step - loss: 0.5815 - mae: 0.5657
Epoch 3/5
516/516 [==============================] - 1s 2ms/step - loss: 0.5440 - mae: 0.5437
Epoch 4/5
516/516 [==============================] - 1s 2ms/step - loss: 0.5221 - mae: 0.5307
Epoch 5/5
129/129 [==============================] - 0s 2ms/step - loss: 0.5205 - mae: 0.5255
Epoch 1/5
516/516 [==============================] - 1s 2ms/step - loss: 0.7432 - mae: 0.6465
Epoch 2/5
516/516 [==============================] - 1s 2ms/step - loss: 0.4956 - mae: 0.5157
Epoch 3/5
516/516 [==============================] - 1s 3ms/step - loss: 0.4683 - mae: 0.4999
Epoch 4/5
516/516 [==============================] - 1s 2ms/step - loss: 0.4590 - mae: 0.4940
Epoch 5/5
129/129 [==============================] - 0s 2ms/step - loss: 0.4666 - mae: 0.4888
Epoch 1/5
516/516 [==============================] - 1s 2ms/

In [ ]:
# Extract best parameters
results[results['MSE'] == results['MSE'].min()]

,Activation,Optimizer,MSE,MAE
15,relu,adam,0.384966,0.437916


# Exercise 2

Select the best optimizer and use it for this exercise. Experiment with different activation functions, including at least sigmoid, tanh, and relu. Rank the activation functions you try. 

In [ ]:
nn_relu = build_nn('relu')

nn_relu.compile(
    optimizer = 'adam',
    loss = 'mse',
    metrics = ['mae']
    )

nn_relu.fit(Z_train,
            y_train,
            epochs = 10
            )

mse, mae = nn_relu.evaluate(Z_test,
                            y_test
                            )

print(f'Test MSE = {round(mse, 3)}, test MAE = {round(mae, 3)}.')

Epoch 1/10
516/516 [==============================] - 1s 2ms/step - loss: 1.1508 - mae: 0.7309
Epoch 2/10
516/516 [==============================] - 1s 2ms/step - loss: 0.4712 - mae: 0.4903
Epoch 3/10
516/516 [==============================] - 1s 2ms/step - loss: 0.4354 - mae: 0.4613
Epoch 4/10
516/516 [==============================] - 1s 2ms/step - loss: 0.3946 - mae: 0.4448
Epoch 5/10
516/516 [==============================] - 1s 2ms/step - loss: 0.3794 - mae: 0.4385
Epoch 6/10
516/516 [==============================] - 1s 2ms/step - loss: 0.3730 - mae: 0.4324
Epoch 7/10
516/516 [==============================] - 1s 2ms/step - loss: 0.3648 - mae: 0.4279
Epoch 8/10
516/516 [==============================] - 1s 2ms/step - loss: 0.3691 - mae: 0.4254
Epoch 9/10
516/516 [==============================] - 1s 2ms/step - loss: 0.3520 - mae: 0.4197
Epoch 10/10
129/129 [==============================] - 0s 2ms/step - loss: 0.3539 - mae: 0.4129
Test MSE = 0.354, test MAE = 0.413.


# Exercise 3

Using your findings, as well as experimenting with more layers, try to minimize the test MSE.

In [ ]:
def build_better_nn(activation):
    your_regression_nn = Sequential([
        Dense(32, activation = activation, input_shape = (8,)), # input_shape = 8 since 8 features
        Dense(64, activation = activation),
        Dense(128, activation = activation),
        Dense(1, activation = 'linear') # linear is used for regression. 1 node since 1 output
        ])

    return your_regression_nn

In [ ]:
nn_final = build_better_nn('relu')

nn_final.compile(
    optimizer = 'adam',
    loss = 'mse',
    metrics = ['mae']
    )

nn_final.fit(Z_train,
             y_train,
             epochs = 10
             )

mse, mae = nn_final.evaluate(Z_test,
                             y_test
                             )

print(f'Final model test MSE = {round(mse, 3)}, test MAE = {round(mae, 3)}.')

Epoch 1/10
516/516 [==============================] - 1s 2ms/step - loss: 0.6948 - mae: 0.5720
Epoch 2/10
516/516 [==============================] - 1s 2ms/step - loss: 0.3869 - mae: 0.4418
Epoch 3/10
516/516 [==============================] - 1s 2ms/step - loss: 0.3609 - mae: 0.4229
Epoch 4/10
516/516 [==============================] - 1s 2ms/step - loss: 0.3336 - mae: 0.4050
Epoch 5/10
516/516 [==============================] - 1s 2ms/step - loss: 0.3239 - mae: 0.3989
Epoch 6/10
516/516 [==============================] - 1s 2ms/step - loss: 0.3117 - mae: 0.3896
Epoch 7/10
516/516 [==============================] - 1s 2ms/step - loss: 0.3038 - mae: 0.3834
Epoch 8/10
516/516 [==============================] - 1s 2ms/step - loss: 0.3016 - mae: 0.3816
Epoch 9/10
516/516 [==============================] - 1s 2ms/step - loss: 0.2928 - mae: 0.3734
Epoch 10/10
129/129 [==============================] - 0s 2ms/step - loss: 0.3283 - mae: 0.4120
Final model test MSE = 0.328, test MAE = 0.412.
